## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for Healthcare Common Procedure Coding System (HCPCS)

- **Model**: `hcpcs_resolver_pipeline_en`
- **Model Description**: This pipeline extracts PROCEDURE entities and maps them to their corresponding Healthcare Common Procedure Coding System (HCPCS) codes using `sbiobert_base_cased_mli` sentence embeddings.

In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "hcpcs-resolver-pipeline-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)


### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    """The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. Hydrochlorothiazide 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.""",
    """Two weeks prior to presentation , she was treated with a five-day course of amoxicillin for a respiratory tract infection . She was on metformin , glipizide , and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG . She had been on dapagliflozin for six months at the time of presentation .The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night , 12 units of insulin lispro with meals , and metformin 1000 mg two times a day.""",
]

sample_text = """Mary received a mechanical prosthetic heart valve in June 2020, and the results were successful. Diabetes screening test performed, revealing abnormal result. She uses infusion pump for diabetes and a CPAP machine for sleep apnea. In 2021, She received a breast prosthesis implant."""

### JSON

In [9]:
input_json_data = {"text": sample_text}
data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"][0])

,ner_chunk,begin,end,ner_label,ner_confidence,code,resolution,all_k_codes,all_k_resolutions,all_k_distances
0,a mechanical prosthetic heart valve,14,48,PROCEDURE,0.83616,G0043,Patients with mechanical prosthetic heart valve,G0043:::C1824:::L8698:::Q0508:::C1764:::C1883::: AV:::V5095:::L8699:::C1899:::L8615:::C1817:::L8630::: RA:::V5273:::Q0507:::L8627:::L8616:::L8628:::L7510:::C1785:::L8618:::L8619:::Q0478:::A4612,"Patients with mechanical prosthetic heart valve:::Generator, cardiac contractility modulation (implantable):::Miscellaneous component, supply or accessory for use with total artificial heart system:::Miscellaneous supply or accessory for use with an implanted ventricular assist device:::Event recorder, cardiac (implantable):::Adapter/extension, pacing lead or neurostimulator lead (implantable):::Item furnished in conjunction with a prosthetic device, prosthetic or orthotic:::Semi-implantable middle ear hearing prosthesis:::Prosthetic implant, not otherwise specified:::Lead, pacemaker/cardioverter-defibrillator combination (implantable):::Headset/headpiece for use with cochlear implant device, replacement:::Septal defect implant system, intracardiac:::Metacarpophalangeal joint implant:::Replacement of a dme, orthotic or prosthetic item:::Assistive listening device, for use with cochlear implant:::Miscellaneous supply or accessory for use with an external ventricular assist device:::Cochlear implant, external speech processor, component, replacement:::Microphone for use with cochlear implant device, replacement:::Cochlear implant, external controller component, replacement:::Repair of prosthetic device, repair or replace minor parts:::Pacemaker, dual chamber, rate-responsive (implantable):::Transmitter cable for use with cochlear implant device or auditory osseointegrated device, replacement:::Cochlear implant, external speech processor and controller, integrated system, replacement:::Power adapter for use with electric or electric/pneumatic ventricular assist device, vehicle type:::Battery cables; replacement for patient-owned ventilator",5.0285:::12.0859:::12.1701:::12.4047:::12.6035:::12.7683:::12.7944:::12.8725:::13.0988:::13.2692:::13.3015:::13.3538:::13.3684:::13.3699:::13.3699:::13.4406:::13.4473:::13.4595:::13.4826:::13.5638:::13.5843:::13.5895:::13.6011:::13.6032:::13.6326
1,infusion pump,168,180,PROCEDURE,0.95785,C1772,"Infusion pump, programmable (implantable)",C1772::: JA:::C1754:::A4220::: SH:::B9004:::S9007:::B9002:::C1887:::A9277:::C2626:::C1729:::E0785:::C1755:::J9031:::E0791:::C1788:::A4353:::A4620::: KD:::K0105:::E0784:::A9278:::A4480:::C1821,"Infusion pump, programmable (implantable):::Administered intravenously:::Catheter, intradiscal:::Refill kit for implantable infusion pump:::Second concurrently administered infusion therapy:::Parenteral nutrition infusion pump, portable:::Ultrafiltration monitor:::Enteral nutrition infusion pump, any type:::Catheter, guiding (may include infusion/perfusion capability):::Transmitter; external, for use with interstitial continuous glucose monitoring system:::Infusion pump, non-programmable, temporary (implantable):::Catheter, drainage:::Implantable intraspinal (epidural/intrathecal) catheter used with implantable infusion pump, replacement:::Catheter, intraspinal:::Bcg (intravesical) per instillation:::Parenteral infusion pump, stationary, single or multi-channel:::Port, indwelling (implantable):::Intermittent urinary catheter, with insertion supplies:::Variable concentration mask:::Drug or biological infused through dme:::Iv hanger, each:::External ambulatory infusion pump, insulin:::Receiver (monitor); external, for use with interstitial continuous glucose monitoring system:::Vabra aspirator:::Interspinous process distraction device (implantable)",9.3487:::10.7028:::11.1157:::11.2839:::11.5007:::11.5293:::11.5971:::11.6919:::11.7318:::11.7456:::11.7574:::11.8079:::11.9601:::12.1554:::12.1567:::12.1791:::12.2640:::12.2825:::12.3008:::12.3038:::12.3786:::12.3827:

### JSON Lines

In [10]:
import json

def create_jsonl(records):

    if isinstance(records, str):
        records = [records]

    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [11]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "a mechanical prosthetic heart valve", "begin": 14, "end": 48, "ner_label": "PROCEDURE", "ner_confidence": "0.83616", "code": "G0043", "resolution": "Patients with mechanical prosthetic heart valve", "all_k_codes": "G0043:::C1824:::L8698:::Q0508:::C1764:::C1883:::   AV:::V5095:::L8699:::C1899:::L8615:::C1817:::L8630:::   RA:::V5273:::Q0507:::L8627:::L8616:::L8628:::L7510:::C1785:::L8618:::L8619:::Q0478:::A4612", "all_k_resolutions": "Patients with mechanical prosthetic heart valve:::Generator, cardiac contractility modulation (implantable):::Miscellaneous component, supply or accessory for use with total artificial heart system:::Miscellaneous supply or accessory for use with an implanted ventricular assist device:::Event recorder, cardiac (implantable):::Adapter/extension, pacing lead or neurostimulator lead (implantable):::Item furnished in conjunction with a prosthetic device, prosthetic or orthotic:::Semi-implantable middle ear hearing prosthesis:::Pr

### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [12]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [13]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [14]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [16]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [17]:
process_s3_json_output_and_save(validation_json_file_name)

{'predictions': [[{'ner_chunk': 'Aspirin',
    'begin': 306,
    'end': 312,
    'ner_label': 'PROCEDURE',
    'ner_confidence': '0.9998',
    'code': 'G8598',
    'resolution': 'Aspirin or another antiplatelet therapy used',
    'all_k_codes': 'G8598:::M1055:::G9435:::G8895:::G6030:::C1765:::A4570:::G6038:::C1769:::C1884:::A4480:::S0518:::E1353:::G6049:::G6034:::A4617:::G6039:::   TD:::   JF:::A9300:::A4670:::S9007:::G8580:::E0945:::M0076',
    'all_k_resolutions': 'Aspirin or another antiplatelet therapy used:::Aspirin or another antiplatelet therapy used:::Aspirin prescribed at discharge:::Oral aspirin or other antithrombotic therapy prescribed:::Amitriptyline:::Adhesion barrier:::Splint:::Salicylate:::Guide wire:::Embolization protective system:::Vabra aspirator:::Sunglasses frames:::Regulator:::Epiandrosterone:::Doxepin:::Mouth piece:::Acetaminophen:::Rn:::Compounded drug:::Exercise equipment:::Automatic blood pressure monitor:::Ultrafiltration monitor:::Antiplatelet medication co

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [20]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "Aspirin", "begin": 306, "end": 312, "ner_label": "PROCEDURE", "ner_confidence": "0.9998", "code": "G8598", "resolution": "Aspirin or another antiplatelet therapy used", "all_k_codes": "G8598:::M1055:::G9435:::G8895:::G6030:::C1765:::A4570:::G6038:::C1769:::C1884:::A4480:::S0518:::E1353:::G6049:::G6034:::A4617:::G6039:::   TD:::   JF:::A9300:::A4670:::S9007:::G8580:::E0945:::M0076", "all_k_resolutions": "Aspirin or another antiplatelet therapy used:::Aspirin or another antiplatelet therapy used:::Aspirin prescribed at discharge:::Oral aspirin or other antithrombotic therapy prescribed:::Amitriptyline:::Adhesion barrier:::Splint:::Salicylate:::Guide wire:::Embolization protective system:::Vabra aspirator:::Sunglasses frames:::Regulator:::Epiandrosterone:::Doxepin:::Mouth piece:::Acetaminophen:::Rn:::Compounded drug:::Exercise equipment:::Automatic blood pressure monitor:::Ultrafiltration monitor:::Antiplatelet medication contraindicated:::Extremity belt/ha

In [21]:
model.delete_model()

INFO:sagemaker:Deleting model with name: hcpcs-resolver-pipeline-en-2024-11-19-18-03-12-294


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

